In [4]:
# Install Required Libraries
# in a new Colab cell
!wget -O sample-bb84.zip https://github.com/aws-samples/sample-BB84-qkd-on-amazon-braket/archive/refs/heads/main.zip
!unzip -q sample-bb84.zip
# cd into the repo folder that was created
%cd sample-BB84-qkd-on-amazon-braket-main
# show files so you can confirm utils/ and the notebook are present
!ls -la

--2025-09-23 03:46:32--  https://github.com/aws-samples/sample-BB84-qkd-on-amazon-braket/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/aws-samples/sample-BB84-qkd-on-amazon-braket/zip/refs/heads/main [following]
--2025-09-23 03:46:32--  https://codeload.github.com/aws-samples/sample-BB84-qkd-on-amazon-braket/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.116.10
Connecting to codeload.github.com (codeload.github.com)|140.82.116.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘sample-bb84.zip’

sample-bb84.zip         [ <=>                ]  21.61K  --.-KB/s    in 0.006s  

2025-09-23 03:46:32 (3.79 MB/s) - ‘sample-bb84.zip’ saved [22126]

/content/sample-BB84-qkd-on-amazon-braket-main
total 44
drw

In [5]:
!pip install --quiet amazon-braket-sdk amazon-braket-default-simulator numpy jupyter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.4/370.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.1/231.1 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.8/537.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [23]:
import base64
import numpy as np

from braket.circuits import noises
from braket.devices import LocalSimulator

from utils.bb84 import initialize_protocol, encode_qubits, measure_qubits, filter_qubits, array_to_string
from utils.golay_code import GolayCode
from utils.secret_utils import convert_to_octets

BIT_FLIP_PROBABILITY = 0.15
NUMBER_OF_QUBITS = 12
ERROR_CORRECTION_CHUNK_SIZE = 12

alice_raw_key = np.array([])
bob_raw_key = np.array([])

In [24]:
# Generate key material until there are 12 bits of raw key

while len(alice_raw_key) < ERROR_CORRECTION_CHUNK_SIZE:

    # For Alice, the important basis is encoding basis.
    encoding_basis_A, states_A, _ = initialize_protocol(NUMBER_OF_QUBITS)

    # Print the initial state of Alice
    sent_bits = array_to_string(states_A)

    # For Bob, the relevant basis is measurement basis.
    _, _, measurement_basis_B = initialize_protocol(NUMBER_OF_QUBITS)

    # Alice encodes the values of her qubits using according bases from `encoding_bases_A`.
    # This is stored as a Qiskit quantum circuit.
    encoded_qubits_A = encode_qubits(NUMBER_OF_QUBITS, states_A, encoding_basis_A)

    # Transmission of encoded qubits to Bob - might add noise!
    noise = noises.BitFlip(probability=BIT_FLIP_PROBABILITY)
    encoded_qubits_A.apply_gate_noise(noise)

    # Bob performs measurement on the received qubits
    measured_circuit = measure_qubits(encoded_qubits_A, measurement_basis_B)
    device = LocalSimulator("braket_dm")
    result = device.run(measured_circuit, shots=1).result()
    measured_bits = list(result.measurements[0])

    # After Bob has measured the qubits, he sends his measurement bases to Alice.
    # She responds to Bob by sending him her encoding bases. Now both parties know both the encoding basis and measurement basis for each qubit.
    # In the key sifting phase, both sides keep only the qubits for which encoding basis and measurement basis were the same.
    alice_raw_key = np.concatenate( (alice_raw_key, filter_qubits(sent_bits, encoding_basis_A, measurement_basis_B)) )
    bob_raw_key = np.concatenate( (bob_raw_key, filter_qubits(measured_bits, encoding_basis_A, measurement_basis_B)) )

In [34]:
alice_raw_key = alice_raw_key[:12]
alice_raw_key

bob_raw_key = bob_raw_key[:12]
bob_raw_key

array([1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.])

In [35]:
error_correcting_code = GolayCode()

generator_matrix = error_correcting_code.get_generator_matrix()
parity_check = error_correcting_code.get_parity_check_matrix()
b_matrix = error_correcting_code.get_b_matrix()

In [36]:
encoded_key_A = np.matmul(generator_matrix, alice_raw_key) % 2
print(f'Key of Alice after encoding is: {encoded_key_A}')
syndrome_A = np.matmul(encoded_key_A, parity_check) % 2
print(f'Syndrom of Alice (should be all zero): {syndrome_A}')
parity_bits = encoded_key_A[12:]
print(f'Information sent to Bob: {parity_bits}')

Key of Alice after encoding is: [1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]
Syndrom of Alice (should be all zero): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Information sent to Bob: [1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]


In [37]:
encoded_key_B = np.concatenate((bob_raw_key, parity_bits))
print(encoded_key_B)
syndrome_B = np.matmul(encoded_key_B, parity_check) % 2
print(syndrome_B)
syndrome_BB = np.matmul(syndrome_B, b_matrix) % 2
syndrome_BB

[1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]
[1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.]


array([0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.])

In [29]:
if syndrome_BB.sum() < 4:
    correction_mask = np.concatenate((syndrome_BB, np.zeros(12,)))
    print(correction_mask)
else:
    print("Decoding failed - more than 3 errors")

[0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [30]:
corrected_key = np.mod(bob_raw_key + correction_mask[:12], 2).astype(int)
corrected_key

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [31]:
corrected_key == alice_raw_key

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [32]:
ASCII_key = base64.b64encode(convert_to_octets(array_to_string(corrected_key))).decode('ascii')
print(ASCII_key)

sw==


In [12]:
!python sample-bb84_with_noise_and_golay.py


Key of Alice after encoding is: [0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1.]
Syndrom of Alice (should be all zero): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Information sent to Bob: [1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1.]
[0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1.]
[1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cA==
